# <center> Assignment #6 (demo).
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
<img src=https://habrastorage.org/webt/-h/ns/aa/-hnsaaifymavmmudwip9imcmk58.jpeg width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [16]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [5]:

DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [6]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [10]:
data.quality.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [14]:
y = data.quality  # you code here
X = data.loc[:, data.columns != 'quality']
X_train, X_holdout, y_train, y_holdout = train_test_split(X,y,test_size=0.3,random_state=17) # you code here

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # you code here
X_holdout_scaled = scaler.transform(X_holdout) # you code here

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [25]:
linreg = LinearRegression() # you code here
model = linreg.fit(X_train_scaled,y_train) # you code here

y1 = model.predict(X_train_scaled)
y2 = model.predict(X_holdout_scaled)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [26]:
print("Mean squared error (train): %.3f" %mean_squared_error(y1,y_train)) # you code here
print("Mean squared error (test): %.3f" %mean_squared_error(y2,y_holdout) )# you code here

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [53]:

linreg_coef = pd.DataFrame(model.coef_,index = data.columns[0:11],columns = ['coef']) 
linreg_coef['abs'] = np.absolute(linreg_coef['coef']) # you code here
linreg_coef.sort_values(by = 'abs') # you code here

,coef,abs
citric acid,-0.000183,0.000183
chlorides,0.008127,0.008127
total sulfur dioxide,0.014304,0.014304
free sulfur dioxide,0.042180,0.042180
sulphates,0.062053,0.062053
fixed acidity,0.097822,0.097822
alcohol,0.129533,0.129533
pH,0.150036,0.150036
volatile acidity,-0.192260,0.192260
residual sugar,0.538164,0.538164


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [61]:
lasso1 = Lasso(alpha=0.01,random_state=17) # you code here
lasso1.fit(X_train_scaled,y_train) # you code her
lasso_y_test = lasso1.predict(X_holdout_scaled)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [62]:
lasso1_coef = pd.DataFrame(lasso1.coef_,index = data.columns[0:11],columns = ['coef'])  # you code here
lasso1_coef['abs'] = lasso1_coef['coef'].apply(lambda x: abs(x)) # you code here
lasso1_coef.sort_values(by = 'abs') # you code here

,coef,abs
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000
total sulfur dioxide,-0.000000,0.000000
chlorides,-0.002747,0.002747
sulphates,0.029722,0.029722
free sulfur dioxide,0.043088,0.043088
pH,0.067277,0.067277
volatile acidity,-0.188479,0.188479
density,-0.235492,0.235492
residual sugar,0.256363,0.256363


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [64]:


alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas = alphas,n_jobs = -1,random_state=17) # you code here
lasso_cv.fit(X_train_scaled,y_train) # you code here

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        n_jobs=-1, random_state=17)

In [65]:
lasso_cv.alpha_

0.0002833096101839324

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [68]:
lasso2 = Lasso(alpha=lasso_cv.alpha_,random_state=17) # you code here
lasso2.fit(X_train_scaled,y_train) # you code her
lasso2_y_test = lasso2.predict(X_holdout_scaled)
lasso2_y_train = lasso2.predict(X_train_scaled)
lasso2_coef = pd.DataFrame(lasso2.coef_,index = data.columns[0:11],columns = ['coef'])  # you code here
lasso2_coef['abs'] = lasso2_coef['coef'].apply(lambda x: abs(x)) # you code here
lasso2_coef.sort_values(by = 'abs') # you code here

,coef,abs
citric acid,-0.000000,0.000000
chlorides,0.006933,0.006933
total sulfur dioxide,0.012969,0.012969
free sulfur dioxide,0.042698,0.042698
sulphates,0.060939,0.060939
fixed acidity,0.093295,0.093295
alcohol,0.137115,0.137115
pH,0.146549,0.146549
volatile acidity,-0.192049,0.192049
residual sugar,0.526883,0.526883


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [70]:
print("Mean squared error (train): %.3f" %mean_squared_error(lasso2_y_train,y_train))# you code here
print("Mean squared error (test): %.3f" %mean_squared_error(lasso2_y_test,y_holdout)) # you code here

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [103]:
forest = RandomForestRegressor(n_jobs=-1,random_state=17) # you code here
forest.fit(X_train,y_train) # you code here
y_test_predict = forest.predict(X_holdout)
y_train_predict = forest.predict(X_train)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [101]:
print("Mean squared error (train): %.3f" %mean_squared_error(y_train_predict,y_train)) # you code here
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(forest,X = X_train_scaled,y = y_train,scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" %mean_squared_error(y_test_predict,y_holdout) )# you code here

Mean squared error (train): 0.054
Mean squared error (cv): 0.408
Mean squared error (test): 0.376


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [111]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'max_features': list(range(6,12))}
forest2 = RandomForestRegressor(n_jobs=-1,random_state=17)
locally_best_forest = GridSearchCV(forest2,param_grid=forest_params,n_jobs=-1,scoring="neg_mean_squared_error",) # you code here
locally_best_forest.fit(X_train_scaled,y_train) # you code here

GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11]},
             scoring='neg_mean_squared_error')

In [113]:
 locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6}, -0.39773288191505934)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [116]:
final_forest = RandomForestRegressor(max_depth=21,max_features=6,random_state = 17)
final_forest.fit(X_train_scaled,y_train)
y_1 = final_forest.predict(X_holdout)
print("Mean squared error (cv): %.3f" % np.sum(np.abs(cross_val_score(locally_best_forest.best_estimator_,X_train_scaled,y_train,scoring='neg_mean_squared_error'))))

print("Mean squared error (test): %.3f" %mean_squared_error(y_holdout,y_1)) # you code here

Mean squared error (cv): 1.989
Mean squared error (test): 0.938


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [127]:
rf_importance = pd.DataFrame(final_forest.feature_importances_,index = data.columns[0:11],columns = ['coef']) 
rf_importance
rf_importance.sort_values(by = 'coef') # you code here

,coef
sulphates,0.059111
fixed acidity,0.061813
citric acid,0.062601
residual sugar,0.072072
chlorides,0.073366
total sulfur dioxide,0.073640
pH,0.073659
density,0.088549
free sulfur dioxide,0.111556
volatile acidity,0.117578


**Make conclusions about the perdormance of the explored 3 models in this particular prediction task.**